In [1]:
def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 100.0 * currentCount / totalCount
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\r[{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print('\n')


def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = len(lst_long) / n
    else:
        totalBatches = len(lst_long) / n + 1
    for i in range(int(totalBatches)):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted

In [9]:
import sys,random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

In [3]:
from sqlalchemy import create_engine
class SteamObject(object):
    
    def config_database_MySQLConnect(self, username,password,host,database):
        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8mb4'.format(username, password, 
                                                                                    host, database))
        return engine
        
    def MySQL_query(self, query):
        df = pd.read_sql_query(query,engine)
        return df
    

In [4]:
# enter your username,password, host and database in mySQL
Steam = SteamObject()
# config database connection
username = ''
password = ''
host = ''
database = ''
engine = Steam.config_database_MySQLConnect(username,password,host,database)

In [5]:
query ='''SELECT app_id, short_description FROM game_steam_app WHERE short_description IS NOT NULL 
            AND type = "game" AND release_date <= CURDATE() AND initial_price IS NOT NULL'''
df_game_description = Steam.MySQL_query(query)
df_game_description.head()

,app_id,short_description
0,1006720,PUZZLE: BIRDS - a logical game in which your t...
1,1006880,"Casual colorful physical puzzle with sweets, j..."
2,1005900,"This is an FPS action game, in which you will ..."
3,1005910,TAKANARIA is a very simple game created with P...
4,1006170,This game is a purely placed game. The player ...


In [24]:
query2 = '''SELECT user_id,app_id, playtime_forever FROM game_steam_user_inventory where playtime_forever <> 0'''
df_game_time = Steam.MySQL_query(query2)
df_game_time.head()

,user_id,app_id,playtime_forever
0,76561198158086086,4000,370
1,76561198158086086,110800,9
2,76561198158086086,250320,507
3,76561198158086086,296470,78
4,76561198158086086,301520,20


In [237]:
class cdfTransformer(object):
    
    transformerSeries = {}
    
    def fitTransform(self,data):
        grouped1 = data.groupby(["app_id","playtime_forever"]).count()
        grouped2 = grouped1.groupby(level=[0]).cumsum()
        grouped3 = grouped2.groupby(level = [0]).max()
        withcdf = grouped2/grouped3
        self.transformerSeries = pd.Series(withcdf['user_id'],index=withcdf.index)
        withcdf_df = withcdf.reset_index(level=[0,1])
        withcdf_df.rename(columns={"user_id":"playtime_cdf"}, inplace=True)
        df = pd.merge(withcdf_df,df_game_time, on=['app_id','playtime_forever'],how='inner')
        df.drop('playtime_forever',axis =1, inplace =True)
        df = df[['user_id','app_id','playtime_cdf']]
        df = df.rename(columns={'playtime_cdf':'rating'})
        return df

In [238]:
df_ratings = cdfTransformer().fitTransform(df_game_time)
df_ratings

,user_id,app_id,rating
0,76561198069986134,10,0.054187
1,76561198062431903,10,0.054187
2,76561198218988347,10,0.054187
3,76561198060984502,10,0.054187
4,76561198056211205,10,0.054187
...,...,...,...
266128,76561198120451477,502550,0.500000
266129,76561198190696911,502550,0.625000
266130,76561198145511975,502550,0.750000
266131,76561198192272162,502550,0.875000


In [236]:
df_game_time.user_id.unique()

array([76561198158086086, 76561198074188133, 76561198058088990, ...,
       76561198126847836, 76561197972183630, 76561198088150884])

In [8]:
tfidf = TfidfVectorizer(strip_accents='unicode',stop_words='english').fit_transform(df_game_description.short_description.tolist())

lst_app_id = df_game_description.app_id.tolist()
dic_recomended = {}
total_count = df_game_description.shape[0]
current_count = 0
for row_index in range(tfidf.shape[0]):
    cosine_similarities = linear_kernel(tfidf[row_index:row_index+1], tfidf).flatten()
    top_related_rows = cosine_similarities.argsort()[-2:-22:-1]
    dic_recomended.update({lst_app_id[row_index]:[lst_app_id[i] for i in top_related_rows]})
    show_work_status(1,total_count,current_count)
    current_count+=1


df_content_based_results = pd.DataFrame.from_dict(dic_recomended, 'index')
df_content_based_results.index.name = 'app_id'
df_content_based_results.reset_index(inplace=True)
df_content_based_results.to_sql('recommended_games_content_based',engine,if_exists='replace', index = False)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [ ]:
## hits/ users
# step 1: 
# step2: see how many hits in first top 20 recommendations for each user

In [258]:
df_content_based_results

,app_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,1006720,962050,909480,775910,470470,896570,912190,970150,389460,566940,...,356250,344230,485660,845150,983520,740220,707320,927670,451670,694610
1,1006880,864570,844160,892880,941790,815950,790860,983280,764560,749340,...,566160,617670,508900,610360,591420,533300,799070,562220,645090,658560
2,1005900,561680,920770,571050,989300,773120,565630,751430,841410,759650,...,765400,975980,567980,750780,574780,988640,443530,746410,754830,748130
3,1005910,909830,925700,797300,796490,944200,972050,415030,557790,862370,...,530240,988890,795940,353540,937140,649450,316830,250540,848090,389170
4,1006170,250660,854190,820380,744260,363960,453980,398150,289220,630170,...,727320,809150,927370,633350,703200,945550,1000300,590600,941620,972940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27586,333640,661330,676490,885790,965970,668770,458710,853550,491470,693790,...,214150,565120,703270,518610,363660,327980,505400,424250,373720,760560
27587,648350,352400,498280,860930,978710,850960,895430,751590,889080,378420,...,834450,602140,687920,450140,756220,801760,113420,953290,528420,356180
27588,834280,380600,522230,557780,798420,660560,468920,594370,766570,645260,...,534910,835510,794180,393430,566420,501080,601940,930150,817200,860930
27589,598330,673610,728730,319780,581220,919690,369290,379600,584280,800530,...,824450,408740,402210,982010,755140,470280,510380,329280,801870,515020


In [256]:
def hit_rate_metric():
    user_id = df_game_time.groupby('user_id').max().reset_index().user_id.tolist()
    app_id = df_game_time.groupby('user_id').max().reset_index().app_id.tolist()
    df = df_ratings.loc[df_ratings['rating']>0.8]
    hit = 0
    for app,user in zip(app_id,user_id):
        recommended_lst = df_content_based_results.loc[df_content_based_results['app_id']==app].values.flatten().tolist()[:10]
        for rec in recommended_lst:
            if rec in df.loc[df['user_id']==user].app_id.unique().tolist():
                hit +=1
    hit_rate = hit/len(user_id)
    return hit_rate
                
        
        
        


In [257]:
hitrate()

0.23649568769859283

In [255]:
df_content_based_results.loc[df_content_based_results['app_id']==app].values.flatten().tolist()[:10]

[730, 273110, 80, 100, 525100, 612520, 286940, 370970, 749800, 292730]